In [17]:
import pandas as pd
import numpy as np

In [47]:
data = pd.read_csv('code_blocks_big.csv', sep='\t')
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,['invite people for the kaggle party']
1,"\ndf_train = pd.read_csv('../input/train.csv'),",['bring in the six packs']
2,"\ndf_train.columns,",['check the decoration']
3,"\ndf_train['SalePrice'].describe(),",['descriptive statistics summary']
4,"\nsns.distplot(df_train['SalePrice']);,",['histogram']


Уникальных тегов:

In [48]:
data['tag'].nunique()

102123

Размер:

In [49]:
data['tag'].shape

(213535,)

### Оставляем по одному комментарию

**Важно:** если у вас в табличке есть колонка 'Unnamed 0:' или вообще какая то лишняя, то дропайте ее до этого цикла (почему-то это сильно влияет на производительность)

Тут из нескольких комментариев выбирается самый короткий (обосновано тем, что тогда он с большей вероятностью встречается часто ака больше похож на метку). Отсекаются пробелы и всякие пустые метки.

In [54]:
from tqdm import tqdm
all_to_del = set(['', "['']", ' ', '  ', "[' ']", '        ', '     ', '    ', '   ', "['', ' ']", '       ', '      ', \
             '             ', '         ', '          ', "']",\
                 "['   ", "['", "[' ", "['  ", "['    ", "['    "])
rows_to_del = []

In [56]:
for i in tqdm(range(data.shape[0])):
    my_string = data.loc[i, 'tag']
    
    if (my_string is not np.nan) and my_string is not None:
        min_len = len(data.loc[i, 'tag']) + 10
        min_el = None
        s = set(list(map(str, my_string.split("', '"))))

        for el in s:
            if el not in all_to_del and len(el) < min_len:
                min_len = len(el)
                min_el = el

        if min_el is not None:
            data.loc[i, 'tag'] = min_el
        else:
            rows_to_del.append(i)
    else:
        rows_to_del.append(i)

100%|██████████| 213535/213535 [00:41<00:00, 5121.12it/s]


Было:

In [57]:
data.shape

(213535, 2)

In [58]:
data = data.drop(rows_to_del)

Осталось:

In [59]:
data.shape

(212459, 2)

In [60]:
data = data.reset_index()

In [62]:
data.drop(columns=['index'], axis=1, inplace=True)
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,['invite people for the kaggle party']
1,"\ndf_train = pd.read_csv('../input/train.csv'),",['bring in the six packs']
2,"\ndf_train.columns,",['check the decoration']
3,"\ndf_train['SalePrice'].describe(),",['descriptive statistics summary']
4,"\nsns.distplot(df_train['SalePrice']);,",['histogram']


________

### Предобработка текста

In [64]:
import re
def clear_tag(tag):
    tag = tag.replace(' ', '_')
    return re.sub('[^A-Za-z0-9_]+', '',tag) # replace spaces with "_" and removes other punctuation

In [82]:
data['tag'] = list(map(clear_tag, data['tag'])) # apply function
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,invite_people_for_the_kaggle_party
1,"\ndf_train = pd.read_csv('../input/train.csv'),",bring_in_the_six_packs
2,"\ndf_train.columns,",check_the_decoration
3,"\ndf_train['SalePrice'].describe(),",descriptive_statistics_summary
6,\nvar = 'GrLivArea'\ndata = pd.concat([df_trai...,scatter_plot_grlivareasaleprice


In [66]:
def clear_first(tag):
    if tag is not np.nan and tag is not None \
            and len(tag) > 0 and tag[0] == '_':
        return tag[1:]
    else:
        return tag

In [81]:
data['tag'] = list(map(clear_first, data['tag'])) # clears first symbol if it's "_"
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,invite_people_for_the_kaggle_party
1,"\ndf_train = pd.read_csv('../input/train.csv'),",bring_in_the_six_packs
2,"\ndf_train.columns,",check_the_decoration
3,"\ndf_train['SalePrice'].describe(),",descriptive_statistics_summary
6,\nvar = 'GrLivArea'\ndata = pd.concat([df_trai...,scatter_plot_grlivareasaleprice


In [69]:
data = data[data['tag'] != '']
data = data[data['tag'] != '_']
data = data[data['tag'] != '__']
data = data[data['tag'] != '___']
data = data[data['tag'] != '____']
data = data[data['tag'] != '_____']

Осталось уже:

In [70]:
data.shape

(211495, 2)

_____________

### Оставим какие-то самые популярные метки

In [74]:
data['tag'].nunique()

76947

In [75]:
from collections import Counter

mc = Counter(data['tag']).most_common(5000)

In [76]:
strange_to_del = []

Если уникальных блоков кода с этим комментарием меньше 100 (естественно стоит эту константу пробовать менять), то удалим эту метку тк это скорее всего копипаст:

Так делаем среди самых популярных 5к тегов чтобы не тратить время на совсем не популярные метки

In [78]:
for el in tqdm(mc):
    if data[data['tag']==el[0]]['code_block'].nunique() < 100:
        strange_to_del.append(el[0])

100%|██████████| 5000/5000 [00:59<00:00, 84.31it/s]


In [79]:
for el in tqdm(strange_to_del):
    data = data[data['tag'] != el]

100%|██████████| 4955/4955 [02:19<00:00, 35.62it/s]


Здесь естественно теряется много данных:

In [83]:
data.shape

(127996, 2)

Заметим, что выкинули 4955 меток из самых популярных 5000, поэтому оставим 45 самых популярных:

In [86]:
Counter(data['tag']).most_common(45)

[('linear_algebra', 3169),
 ('specify_none_if_want_to_read_whole_file', 2102),
 ('check_your_answer', 591),
 ('model', 573),
 ('create_x', 559),
 ('your_code_here', 440),
 ('fit_rfmodelonfulldata_on_all_data_from_the_training_data', 440),
 ('preprocessing', 420),
 ('feature_engineering', 411),
 ('just_uncomment_them', 404),
 ('submission', 373),
 ('load_data', 356),
 ('random_forest', 322),
 ('training', 279),
 ('prediction', 268),
 ('eda', 241),
 ('check_your_answers', 226),
 ('visualization', 213),
 ('test', 207),
 ('train', 206),
 ('data', 203),
 ('data_preprocessing', 198),
 ('plot', 197),
 ('logistic_regression', 196),
 ('imports', 195),
 ('predict', 184),
 ('data_cleaning', 183),
 ('model_building', 180),
 ('data_visualization', 176),
 ('xgboost', 173),
 ('exploratory_data_analysis', 169),
 ('data_exploration', 160),
 ('decision_tree', 149),
 ('modelling', 144),
 ('importing_libraries', 141),
 ('import_data', 137),
 ('import_libraries', 134),
 ('input', 132),
 ('title', 126),
 ('

____

Здесь еще одна небольшая проверка, чтобы не было копипаста, потому что комментарий в 75 строк совпавший так много раз явно коппаст:

In [88]:
al_r_data = data[data['tag'].str.len() < 75]

In [89]:
print(al_r_data.shape)
print(al_r_data['tag'].nunique())

(96853, 2)
52842


In [91]:
Counter(al_r_data['tag']).most_common(45)

[('linear_algebra', 3169),
 ('specify_none_if_want_to_read_whole_file', 2102),
 ('check_your_answer', 591),
 ('model', 573),
 ('create_x', 559),
 ('your_code_here', 440),
 ('fit_rfmodelonfulldata_on_all_data_from_the_training_data', 440),
 ('preprocessing', 420),
 ('feature_engineering', 411),
 ('just_uncomment_them', 404),
 ('submission', 373),
 ('load_data', 356),
 ('random_forest', 322),
 ('training', 279),
 ('prediction', 268),
 ('eda', 241),
 ('check_your_answers', 226),
 ('visualization', 213),
 ('test', 207),
 ('train', 206),
 ('data', 203),
 ('data_preprocessing', 198),
 ('plot', 197),
 ('logistic_regression', 196),
 ('imports', 195),
 ('predict', 184),
 ('data_cleaning', 183),
 ('model_building', 180),
 ('data_visualization', 176),
 ('xgboost', 173),
 ('exploratory_data_analysis', 169),
 ('data_exploration', 160),
 ('decision_tree', 149),
 ('modelling', 144),
 ('importing_libraries', 141),
 ('import_data', 137),
 ('import_libraries', 134),
 ('input', 132),
 ('title', 126),
 ('

In [93]:
# all_to_del - задан вначале ноутбука

In [94]:
for to_del in all_to_del:
    al_r_data = al_r_data[al_r_data['tag'] != to_del]

In [95]:
print(al_r_data.shape)
print(al_r_data['tag'].nunique())

(96853, 2)
52842


In [99]:
num_tags_to_drop = al_r_data['tag'].nunique() - 45
num_tags_to_drop

52797

Выкидываю самые непопулярные num_tags_to_drop меток:

In [101]:
least_c_tags = set(Counter(al_r_data['tag']).most_common()[-num_tags_to_drop:])

In [104]:
for el in tqdm(least_c_tags):
    al_r_data = al_r_data[al_r_data['tag'] != el[0]]

100%|██████████| 52797/52797 [09:08<00:00, 96.27it/s] 


Смотрим что осталось:

In [105]:
print(al_r_data.shape)
print(al_r_data['tag'].nunique())

(15684, 2)
45


In [107]:
ready_data = \
al_r_data.reset_index().drop(columns=['index'], axis=1)
#ready_data = ready_data.drop(columns=['level_0'], axis=1)

In [119]:
Counter(ready_data['tag']).most_common()[:10]

[('linear_algebra', 3169),
 ('specify_none_if_want_to_read_whole_file', 2102),
 ('check_your_answer', 591),
 ('model', 573),
 ('create_x', 559),
 ('your_code_here', 440),
 ('fit_rfmodelonfulldata_on_all_data_from_the_training_data', 440),
 ('preprocessing', 420),
 ('feature_engineering', 411),
 ('just_uncomment_them', 404)]

In [117]:
Counter(ready_data['tag']).most_common()[-10:]

[('import_data', 137),
 ('import_libraries', 134),
 ('input', 132),
 ('title', 126),
 ('confusion_matrix', 124),
 ('modeling', 123),
 ('linear_regression', 121),
 ('read_data', 117),
 ('train_model', 116),
 ('feature_selection', 106)]

**Важное замечение**

Тут возникают вопросы например почему 440 комментариев 'fit_rfmodelonfulldata_on_all_data_from_the_training_data', тк казалось бы это очень специфичный комментарий. 

У меня есть подозрение, что идет копипаст 1 строки с комментарием, а затем что-то немного меняется и поэтому показывает что разных комментариев 100+ с этим тегом

In [120]:
ready_data[ready_data['tag'] == 'fit_rfmodelonfulldata_on_all_data_from_the_training_data']['code_block'].nunique()

158

__________________

Соотношения train, test, val = 8.5 / 1.5 / 1.5

Поделим нацело:

In [128]:
(ready_data.shape[0] - 9) / 11

1425.0

In [812]:
18366 - 18359

7

In [129]:
ready_data = ready_data.drop(ready_data.tail(9).index)

In [130]:
ready_data.shape[0] / 11

1425.0

То есть размер трейна:

In [132]:
1425.0 * 8

11400.0

Стратифайдом делю на трейн, тест, сплит

In [133]:
# https://stackoverflow.com/questions/40829137/stratified-train-validation-test-split-in-scikit-learn

from sklearn.model_selection import StratifiedShuffleSplit
SEED = 42001

split = StratifiedShuffleSplit(n_splits=1, test_size=3/11, random_state=SEED)
for train_index, test_valid_index in split.split(\
                                                 ready_data.drop('tag', axis=1), \
                                                 ready_data['tag']\
                                                ):
    train_set = ready_data.iloc[train_index]
    test_valid_set = ready_data.iloc[test_valid_index]

split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
for test_index, valid_index in split2.split(test_valid_set, test_valid_set['tag']):
    test_set = test_valid_set.iloc[test_index]
    valid_set = test_valid_set.iloc[valid_index]

In [134]:
train_set = train_set.reset_index()
test_set = test_set.reset_index()
valid_set = valid_set.reset_index()

______

Далее все '\n' заменяются на '\n\t', а также вначале каждого сниппета приписывается

'def tag():\n\t'

Каждый сниппет записывается в **отдельный файл** с названием номер_итерации.py в каталог трей, тест или валидейшн.

In [818]:
def add_tabs(exmpl):
    new_exmpl = str()
    for i in range(len(exmpl)):
        if (exmpl[i]=='\n'):
            new_exmpl += '\n\t'
        else:
            new_exmpl += exmpl[i]
    return new_exmpl

train:

In [822]:
for i in range(train_set.shape[0]):
        exmpl = train_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(train_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_train/func'+str(i)+'.py', 'w'))

test:

In [823]:
for i in range(test_set.shape[0]):
        exmpl = test_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(test_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_test/func'+str(i)+'.py', 'w'))

valid:

In [824]:
for i in range(valid_set.shape[0]):
        exmpl = valid_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(valid_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_val/func'+str(i)+'.py', 'w'))

_______

Предобработка для просто 30 строк кода посложнее, пока ее нет.